### **0. Set-up**

In [3]:
# Import libraries and utils
%run '../../utils.ipynb'
%run '../../utils_structured.py'

In [4]:
# Get api key
load_dotenv()
OPENROUTER_API_KEY = os.getenv('OPENROUTER_API_KEY')

# Set client
client = OpenAI(
    base_url="https://openrouter.ai/api/v1",
    api_key=OPENROUTER_API_KEY,
)

In [5]:
# Load dataframe
en_simlex = pd.read_csv("../../../data/dataset/cleaned-en-simlex-999.csv")

# Select subset
en_simlex = en_simlex.iloc[666:999]

# Convert to tuple
tuples_list = list(zip(en_simlex['word1'], en_simlex['word2']))

In [6]:
# Show results
tuples_list

[('boy', 'soldier'),
 ('belly', 'abdomen'),
 ('guy', 'girl'),
 ('bed', 'chair'),
 ('clothes', 'jacket'),
 ('gun', 'knife'),
 ('tin', 'metal'),
 ('bottle', 'container'),
 ('hen', 'turkey'),
 ('meat', 'bread'),
 ('arm', 'bone'),
 ('neck', 'spine'),
 ('apple', 'lemon'),
 ('agony', 'grief'),
 ('assignment', 'task'),
 ('night', 'dawn'),
 ('dinner', 'soup'),
 ('calf', 'bull'),
 ('snow', 'storm'),
 ('nail', 'hand'),
 ('dog', 'horse'),
 ('arm', 'neck'),
 ('ball', 'glove'),
 ('flu', 'fever'),
 ('fee', 'salary'),
 ('nerve', 'brain'),
 ('beast', 'animal'),
 ('dinner', 'chicken'),
 ('girl', 'maid'),
 ('child', 'boy'),
 ('alcohol', 'wine'),
 ('nose', 'mouth'),
 ('street', 'car'),
 ('bell', 'door'),
 ('box', 'hat'),
 ('belief', 'impression'),
 ('bias', 'opinion'),
 ('attention', 'awareness'),
 ('anger', 'mood'),
 ('elegance', 'style'),
 ('beauty', 'age'),
 ('book', 'theme'),
 ('friend', 'mother'),
 ('vitamin', 'iron'),
 ('car', 'factory'),
 ('pact', 'condition'),
 ('chapter', 'choice'),
 ('arithmeti

### **1. Define and Evaluate Parameters**

In [7]:
prompt = ("Rate the semantic similarity of the word pair: [('{word1}'), ('{word2}')] on a scale from 0 to 10, "
          "where 0 represents no semantic similarity, and 10 represents perfect semantic similarity. "
          "Use two decimals. Do not provide additional explanations or context.")

In [8]:
# Define model
model = "mistralai/mistral-7b-instruct-v0.3"

# Set sample size
sample_size = 15

# Delay between individual API calls
delay = 3.0

# Define number of sublists
n_sublists = 333

In [9]:
# Split list
chunks = split_into_n_lists(tuples_list, n_sublists)

# Count number of lists
print(len(chunks))

333


In [10]:
# Print prompts for each chunk
print_prompts_single(chunks, sample_size, prompt)

Rate the semantic similarity of the word pair: [('boy'), ('soldier')] on a scale from 0 to 10, where 0 represents no semantic similarity, and 10 represents perfect semantic similarity. Use two decimals. Do not provide additional explanations or context.
Rate the semantic similarity of the word pair: [('boy'), ('soldier')] on a scale from 0 to 10, where 0 represents no semantic similarity, and 10 represents perfect semantic similarity. Use two decimals. Do not provide additional explanations or context.
Rate the semantic similarity of the word pair: [('boy'), ('soldier')] on a scale from 0 to 10, where 0 represents no semantic similarity, and 10 represents perfect semantic similarity. Use two decimals. Do not provide additional explanations or context.
Rate the semantic similarity of the word pair: [('boy'), ('soldier')] on a scale from 0 to 10, where 0 represents no semantic similarity, and 10 represents perfect semantic similarity. Use two decimals. Do not provide additional explanati

In [11]:
# Load encoding
encoding = tiktoken.get_encoding("cl100k_base")

# Count tokens per chunk
token_counts = count_tokens_with_tiktoken_single(chunks, prompt)

# Show results
print("Token counts for each formatted prompt:", token_counts)

Token counts for each formatted prompt: [54, 56, 54, 53, 55, 53, 53, 54, 54, 54, 53, 54, 54, 55, 53, 54, 54, 54, 53, 54, 53, 53, 54, 54, 53, 54, 54, 55, 53, 53, 54, 54, 53, 53, 53, 54, 54, 54, 54, 55, 54, 53, 53, 54, 53, 54, 53, 55, 53, 53, 53, 54, 54, 56, 54, 54, 53, 54, 54, 54, 54, 53, 55, 53, 53, 53, 54, 55, 54, 54, 53, 55, 54, 55, 53, 53, 53, 55, 56, 55, 54, 53, 55, 54, 53, 53, 53, 54, 54, 54, 54, 54, 56, 53, 55, 53, 53, 53, 53, 54, 54, 55, 53, 53, 54, 54, 55, 56, 55, 54, 54, 53, 54, 53, 55, 55, 55, 53, 54, 54, 54, 53, 53, 53, 53, 53, 53, 53, 53, 55, 56, 53, 54, 53, 53, 53, 54, 54, 55, 54, 54, 54, 54, 55, 53, 53, 54, 54, 53, 54, 54, 54, 54, 55, 54, 54, 53, 53, 54, 53, 53, 53, 53, 54, 54, 54, 53, 54, 54, 53, 54, 53, 54, 54, 55, 53, 53, 53, 53, 53, 54, 53, 53, 53, 54, 53, 56, 53, 53, 56, 53, 54, 53, 53, 53, 54, 53, 53, 54, 54, 55, 53, 54, 53, 53, 53, 55, 55, 55, 54, 55, 55, 55, 56, 54, 54, 53, 55, 53, 53, 55, 53, 54, 55, 55, 54, 54, 54, 54, 54, 53, 53, 53, 54, 54, 54, 55, 53, 54, 53,

In [12]:
# Max RPD = 10.000
len(token_counts*15)

4995

In [13]:
# Number of total tokens
print(sum(token_counts))

17922


### **2. Extract Data**

In [12]:
# Get results from API
response = get_responses_single_structured(prompt, chunks, model, sample_size, delay, client)

Processing: 100%|██████████| 4995/4995 [6:45:27<00:00,  4.87s/chunk]     

Total time taken: 24328.00 seconds


In [ ]:
# Define filepath
file_path = '../../../data/mistral-7b-instruct-v0.3/response/en/f9-3.json'

# Check if file already exists
if not os.path.exists(file_path):
    with open(file_path, 'w') as f:
        json.dump(response, f)
        print("File saved successfully.")
else:
    print("File already exists. JSON was not saved to prevent overwriting.")

File saved successfully.


### **3. Process and Inspect Data**

In [16]:
# Process data into dictionary
data_dict = process_responses_structured(response)

# Check for values higher/lower then sample size
higher_lower_samples = {key: value for key, value in data_dict.items() if len(value) < sample_size or len(value) > sample_size}

# Show results
print(higher_lower_samples)

{}


In [17]:
# Print duplicate word pairs
print_duplicate_word_pairs(en_simlex, data_dict)

Empty DataFrame
Columns: [Combined_Columns]
Index: []
Empty DataFrame
Columns: [Combined_Columns]
Index: []


In [18]:
# Convert dict to Pandas DataFrame
df = create_dataframe(data_dict)

# Show results
df

,word1,word2,similarity_score_1,similarity_score_2,similarity_score_3,similarity_score_4,similarity_score_5,similarity_score_6,similarity_score_7,similarity_score_8,similarity_score_9,similarity_score_10,similarity_score_11,similarity_score_12,similarity_score_13,similarity_score_14,similarity_score_15
0,boy,soldier,4.25,3.25,3.25,5.25,5.45,5.25,5.25,4.50,3.25,4.25,4.20,4.23,6.25,4.25,3.25
1,belly,abdomen,9.25,8.50,8.95,9.75,9.50,9.80,9.50,9.50,9.50,9.50,9.50,9.25,8.50,9.80,9.50
2,guy,girl,7.22,7.22,7.25,7.25,8.50,7.25,7.80,7.67,6.67,7.80,7.89,7.83,7.50,7.22,7.67
3,bed,chair,6.25,1.00,7.25,7.50,6.25,6.67,6.25,4.20,7.50,1.00,7.50,7.50,6.25,6.67,7.25
4,clothes,jacket,8.75,8.50,8.75,8.50,8.75,8.50,8.50,8.50,8.67,8.50,8.75,8.50,8.50,8.50,8.50
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
328,join,acquire,2.50,2.34,3.45,2.34,2.34,3.50,2.34,3.25,2.50,2.34,1.23,2.50,2.50,2.50,2.22
329,send,attend,1.23,1.23,1.67,1.23,1.23,1.23,1.23,0.20,2.34,1.23,0.00,0.20,1.23,1.23,1.23
330,gather,attend,6.25,6.66,5.50,6.50,5.25,6.25,6.25,6.25,6.45,6.50,4.50,5.60,6.50,6.50,5.20
331,absorb,withdraw,1.23,1.23,2.34,1.23,1.23,2.34,0.20,2.34,1.23,2.34,2.45,1.23,1.67,2.50,2.34


In [19]:
# Count null values
count_null_values = df.isnull().sum()

# Show results
print("Null value counts per column:", count_null_values)

Null value counts per column: word1                  0
word2                  0
similarity_score_1     0
similarity_score_2     0
similarity_score_3     0
similarity_score_4     0
similarity_score_5     0
similarity_score_6     0
similarity_score_7     0
similarity_score_8     0
similarity_score_9     0
similarity_score_10    0
similarity_score_11    0
similarity_score_12    0
similarity_score_13    0
similarity_score_14    0
similarity_score_15    0
dtype: int64


In [20]:
# Check for rows with at least one null value
rows_with_null = df[df.isnull().any(axis=1)]

# Show results
rows_with_null

,word1,word2,similarity_score_1,similarity_score_2,similarity_score_3,similarity_score_4,similarity_score_5,similarity_score_6,similarity_score_7,similarity_score_8,similarity_score_9,similarity_score_10,similarity_score_11,similarity_score_12,similarity_score_13,similarity_score_14,similarity_score_15


### **4. Fix Faulty Word Pairs**

In [21]:
# ...

### **5. Export Data**

In [22]:
# Define file_path
file_path = '../../../data/mistral-7b-instruct-v0.3/processed/en/f9-3.csv'

# Check if file already exists
if not os.path.exists(file_path):
    df.to_csv(file_path, index=False)
    print("File saved successfully.")
else:
    print("File already exists. Dataframe was not saved to prevent overwriting.")

File saved successfully.
